# Quantum Optimization Fundamentals for Food Production

## Learning Objectives
By the end of this notebook, you will:
1. Understand quantum optimization principles applied to agriculture
2. Learn how food production optimization maps to quantum problems
3. Implement the multi-objective optimization framework used in OQI_Project
4. Convert food allocation problems to QUBO formulations
5. Understand quantum advantage in agricultural resource allocation

## Real-World Context: Sustainable Food Production
This tutorial teaches quantum optimization through the lens of **food production optimization**, the core problem solved in the OQI_Project. We optimize:
- **F farms** producing **C different foods**
- **Multi-objective scoring**: nutritional value, sustainability, affordability, nutrient density, environmental impact
- **Constraints**: land availability, food group diversity, minimum utilization

## Prerequisites
- Basic linear algebra
- Python programming
- Elementary understanding of optimization problems

---

In [ ]:
# Essential imports for quantum food production optimization
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import expm
import itertools
from typing import List, Tuple, Dict, Any, Optional
from enum import Enum
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Define the optimization objectives used in OQI_Project
class OptimizationObjective(Enum):
    """Types of optimization objectives for food production."""
    NUTRITIONAL_VALUE = "nutritional_value"
    NUTRIENT_DENSITY = "nutrient_density"
    ENVIRONMENTAL_IMPACT = "environmental_impact"
    AFFORDABILITY = "affordability"
    SUSTAINABILITY = "sustainability"

print("✓ All imports successful!")
print("Ready to explore quantum optimization for food production!")
print(f"Optimization objectives: {[obj.value for obj in OptimizationObjective]}")

## 1. Food Production as a Quantum Optimization Problem

### Why Quantum Computing for Agricultural Optimization?

Food production optimization involves complex multi-objective decisions with numerous constraints:
- **F farms** × **C foods** = **F×C binary decisions** (which foods to grow where)
- **Multi-objective scoring** across 5 dimensions simultaneously
- **Complex constraints** (land, diversity, sustainability)

Quantum computers can potentially explore these exponentially large solution spaces through **superposition** and find optimal allocations through **quantum interference**.

### The Food Production Problem Structure

In the OQI_Project, we optimize:

**Decision Variables:**
- $y_{fc} \in \{0,1\}$: Binary decision to grow food $c$ on farm $f$
- $x_{fc} \geq 0$: Continuous area (hectares) of food $c$ on farm $f$

**Objective Function:**
$$\max \sum_{f,c} \left(\sum_{obj} w_{obj} \cdot score_{c,obj}\right) \cdot x_{fc}$$

Where $obj \in \{nutritional\_value, nutrient\_density, affordability, sustainability, environmental\_impact\}$

### Quantum Mapping: From Farms to Qubits

Each binary decision $y_{fc}$ maps to a qubit state:
- $|0\rangle$: Don't grow food $c$ on farm $f$
- $|1\rangle$: Grow food $c$ on farm $f$

A complete solution is an $F \times C$ quantum state representing all farm-food combinations.

---

## 🔧 Exercise 1: Modeling Food Production Decisions

Let's implement the core data structures used in food production optimization, starting with a simplified version of the OQI_Project food system.

In [ ]:
from typing import List, Dict
import numpy as np
from enum import Enum

class OptimizationObjective(Enum):
    """Objectives for optimizing food production."""
    NUTRITIONAL_VALUE = "nutritional_value"
    NUTRIENT_DENSITY = "nutrient_density"
    ENVIRONMENTAL_IMPACT = "environmental_impact"
    AFFORDABILITY = "affordability"
    SUSTAINABILITY = "sustainability"

class SimpleFoodSystem:
    """A simplified food production system for learning quantum optimization."""
    
    def __init__(self, farms: List[str], foods: Dict[str, Dict[str, float]]):
        self.farms = farms
        self.foods = foods
        self.F = len(farms)  # Number of farms
        self.C = len(foods)  # Number of foods
        self.n_decisions = self.F * self.C  # Total binary decisions
        
        # Initialize solution vector (F×C binary decisions)
        self.solution = np.zeros(self.n_decisions, dtype=int)
    
    def get_decision_index(self, farm_idx: int, food_idx: int) -> int:
        """Convert (farm, food) indices to linear decision index."""
        return farm_idx * self.C + food_idx
    
    def set_decision(self, farm: str, food: str, value: int):
        """Set whether to grow a specific food on a specific farm."""
        farm_idx = self.farms.index(farm)
        food_idx = list(self.foods.keys()).index(food)
        idx = self.get_decision_index(farm_idx, food_idx)
        self.solution[idx] = value
    
    def get_decision(self, farm: str, food: str) -> int:
        """Get whether a specific food is grown on a specific farm."""
        farm_idx = self.farms.index(farm)
        food_idx = list(self.foods.keys()).index(food)
        idx = self.get_decision_index(farm_idx, food_idx)
        return self.solution[idx]
    
    def calculate_objective(self, weights: Dict[str, float]) -> float:
        """Calculate the weighted multi-objective score."""
        total_score = 0.0
        food_names = list(self.foods.keys())
        
        for farm_idx, farm in enumerate(self.farms):
            for food_idx, food in enumerate(food_names):
                if self.get_decision(farm, food) == 1:
                    # Calculate weighted score for this food
                    food_score = 0.0
                    for obj in OptimizationObjective:
                        obj_value = self.foods[food].get(obj.value, 0.0)
                        weight = weights.get(obj.value, 0.0)
                        
                        if obj == OptimizationObjective.ENVIRONMENTAL_IMPACT:
                            food_score -= weight * obj_value  # Minimize environmental impact
                        else:
                            food_score += weight * obj_value  # Maximize other objectives
                    
                    total_score += food_score
        
        return total_score
    
    def __str__(self) -> str:
        """String representation showing current farm-food allocations."""
        result = []
        for farm in self.farms:
            farm_foods = []
            for food in self.foods.keys():
                if self.get_decision(farm, food) == 1:
                    farm_foods.append(food)
            if farm_foods:
                result.append(f"{farm}: {', '.join(farm_foods)}")
        return "\n".join(result) if result else "No allocations"

# Create a test food system based on OQI_Project structure
test_farms = ['Farm1', 'Farm2', 'Farm3']
test_foods = {
    'Wheat': {
        'nutritional_value': 0.7,
        'nutrient_density': 0.6,
        'environmental_impact': 0.3,
        'affordability': 0.8,
        'sustainability': 0.7
    },
    'Rice': {
        'nutritional_value': 0.6,
        'nutrient_density': 0.5,
        'environmental_impact': 0.4,
        'affordability': 0.9,
        'sustainability': 0.6
    },
    'Soybeans': {
        'nutritional_value': 0.9,
        'nutrient_density': 0.8,
        'environmental_impact': 0.2,
        'affordability': 0.6,
        'sustainability': 0.8
    }
}

# Test the implementation
print("Testing SimpleFoodSystem:")
food_system = SimpleFoodSystem(test_farms, test_foods)
print(f"System dimensions: {food_system.F} farms × {food_system.C} foods = {food_system.n_decisions} decisions")

# Set some example allocations
food_system.set_decision('Farm1', 'Wheat', 1)
food_system.set_decision('Farm2', 'Rice', 1)
food_system.set_decision('Farm3', 'Soybeans', 1)

print(f"\nCurrent allocations:\n{food_system}")

# Calculate objective with equal weights
equal_weights = {obj.value: 0.2 for obj in OptimizationObjective}
objective_value = food_system.calculate_objective(equal_weights)
print(f"\nObjective value (equal weights): {objective_value:.4f}")

### 🎯 Challenge 1: Food Allocation Decisions

Your task is to implement a method that evaluates different food allocation strategies by calculating weighted objectives.

**Challenge**: Implement the `evaluate_allocation_strategy` method that tests different farm-food combinations and finds the best allocation strategy.

In [ ]:
def evaluate_allocation_strategy(food_system: SimpleFoodSystem, 
                                strategy_weights: Dict[str, float]) -> Tuple[str, float]:
    """Evaluate different food allocation strategies and find the best one.
    
    This simulates the quantum exploration of different allocation possibilities.
    
    Args:
        food_system: The food production system
        strategy_weights: Weights for different optimization objectives
    
    Returns:
        Tuple of (best_allocation_description, best_score)
    
    TODO: Implement this method!
    Hint: 
    1. Try different combinations of farm-food allocations
    2. Calculate the weighted objective score for each
    3. Return the allocation with the highest score
    """
    best_score = float('-inf')
    best_allocation = "No allocation"
    
    # TODO: Try different allocation strategies
    # Example strategies to test:
    # 1. Each farm grows one type of food
    # 2. All farms grow the same food
    # 3. Diversified allocation (each farm grows multiple foods)
    
    strategies = [
        # Strategy 1: Monoculture per farm
        [("Farm1", "Wheat"), ("Farm2", "Rice"), ("Farm3", "Soybeans")],
        
        # Strategy 2: All wheat
        [("Farm1", "Wheat"), ("Farm2", "Wheat"), ("Farm3", "Wheat")],
        
        # Strategy 3: All soybeans (high nutrition)
        [("Farm1", "Soybeans"), ("Farm2", "Soybeans"), ("Farm3", "Soybeans")],
        
        # TODO: Add your own strategies here!
    ]
    
    for i, strategy in enumerate(strategies):
        # Reset the food system
        food_system.solution = np.zeros(food_system.n_decisions, dtype=int)
        
        # Apply the strategy
        for farm, food in strategy:
            food_system.set_decision(farm, food, 1)
        
        # TODO: Calculate the score for this strategy
        score = 0.0  # Replace with your implementation
        
        # TODO: Update best_score and best_allocation if this is better
        
        strategy_description = f"Strategy {i+1}: {', '.join([f'{f}→{c}' for f, c in strategy])}"
        print(f"{strategy_description}: Score = {score:.4f}")
    
    return best_allocation, best_score

# Test your implementation
def test_allocation_strategy():
    """Test the allocation strategy evaluation."""
    food_system = SimpleFoodSystem(test_farms, test_foods)
    
    # Focus on sustainability and nutrition
    sustainability_weights = {
        'nutritional_value': 0.3,
        'nutrient_density': 0.2,
        'environmental_impact': 0.1,  # Lower weight since we minimize this
        'affordability': 0.2,
        'sustainability': 0.3
    }
    
    best_allocation, best_score = evaluate_allocation_strategy(food_system, sustainability_weights)
    print(f"\n🏆 Best allocation: {best_allocation}")
    print(f"🏆 Best score: {best_score:.4f}")

# Uncomment to test when you've implemented the method:
# test_allocation_strategy()

### 💡 Solution for Challenge 1: Food Allocation Strategy

Click below to reveal the solution:

In [ ]:
# SOLUTION - Don't peek until you've tried!
def evaluate_allocation_strategy_solution(food_system: SimpleFoodSystem, 
                                        strategy_weights: Dict[str, float]) -> Tuple[str, float]:
    """Evaluate different food allocation strategies and find the best one."""
    best_score = float('-inf')
    best_allocation = "No allocation"
    
    strategies = [
        # Strategy 1: Monoculture per farm (diversification)
        [("Farm1", "Wheat"), ("Farm2", "Rice"), ("Farm3", "Soybeans")],
        
        # Strategy 2: All wheat (affordable but less nutrition)
        [("Farm1", "Wheat"), ("Farm2", "Wheat"), ("Farm3", "Wheat")],
        
        # Strategy 3: All soybeans (high nutrition and sustainability)
        [("Farm1", "Soybeans"), ("Farm2", "Soybeans"), ("Farm3", "Soybeans")],
        
        # Strategy 4: All rice (affordable but higher environmental impact)
        [("Farm1", "Rice"), ("Farm2", "Rice"), ("Farm3", "Rice")],
        
        # Strategy 5: Mixed strategy - focus on soybeans and wheat
        [("Farm1", "Soybeans"), ("Farm2", "Wheat"), ("Farm3", "Soybeans")],
    ]
    
    for i, strategy in enumerate(strategies):
        # Reset the food system
        food_system.solution = np.zeros(food_system.n_decisions, dtype=int)
        
        # Apply the strategy
        for farm, food in strategy:
            food_system.set_decision(farm, food, 1)
        
        # Calculate the score for this strategy
        score = food_system.calculate_objective(strategy_weights)
        
        # Update best if this is better
        if score > best_score:
            best_score = score
            best_allocation = f"Strategy {i+1}: {', '.join([f'{f}→{c}' for f, c in strategy])}"
        
        strategy_description = f"Strategy {i+1}: {', '.join([f'{f}→{c}' for f, c in strategy])}"
        print(f"{strategy_description}: Score = {score:.4f}")
    
    return best_allocation, best_score

# Replace with solution and test
evaluate_allocation_strategy = evaluate_allocation_strategy_solution
test_allocation_strategy()

## 2. Food Production Constraints as Quantum Problems

Real food production optimization involves complex constraints that map naturally to quantum optimization problems. Let's implement the constraint modeling used in the OQI_Project.

### Key Constraints in Food Production:
1. **Land Allocation**: Each farm has limited land area
2. **Food Diversity**: Must grow different types of foods
3. **Minimum Production**: Each food type needs minimum production
4. **Resource Limits**: Water, nutrients, labor constraints

### Mathematical Foundation

For F farms and C foods, we have binary decisions $y_{fc} \in \{0,1\}$:
- **Objective**: $\max \sum_{f,c} score_{fc} \cdot y_{fc}$
- **Land constraint**: $\sum_c y_{fc} \leq capacity_f$ for each farm $f$
- **Diversity constraint**: $\sum_f y_{fc} \geq 1$ for each food $c$

### QUBO Formulation (Quadratic Unconstrained Binary Optimization)

We convert constraints to penalty terms:
$$H = -\sum_{f,c} score_{fc} \cdot y_{fc} + \lambda_1 \sum_f \text{penalty}_{land}(f) + \lambda_2 \sum_c \text{penalty}_{diversity}(c)$$

This maps directly to quantum optimization where finding the ground state gives the optimal allocation.

In [ ]:
from typing import List, Dict
import numpy as np

class FoodProductionQUBO:
    """QUBO formulation for food production optimization.
    
    This class implements the quantum optimization formulation used in OQI_Project.
    """
    
    def __init__(self, farms: List[str], foods: Dict[str, Dict[str, float]]):
        self.farms = farms
        self.foods = foods
        self.F = len(farms)
        self.C = len(foods)
        self.n_variables = self.F * self.C
        
        # QUBO matrix Q (symmetric)
        self.Q = np.zeros((self.n_variables, self.n_variables))
        
        # Variable mapping: farm_idx * C + food_idx
        self.food_names = list(foods.keys())
    
    def get_variable_index(self, farm_idx: int, food_idx: int) -> int:
        """Get linear index for binary variable y_{farm,food}."""
        return farm_idx * self.C + food_idx
    
    def set_objective_weights(self, weights: Dict[str, float]):
        """Set the objective function weights (diagonal terms in QUBO)."""
        for farm_idx in range(self.F):
            for food_idx in range(self.C):
                var_idx = self.get_variable_index(farm_idx, food_idx)
                
                # Calculate weighted score for this farm-food combination
                food_name = self.food_names[food_idx]
                food_data = self.foods[food_name]
                
                score = 0.0
                for obj in OptimizationObjective:
                    obj_value = food_data.get(obj.value, 0.0)
                    weight = weights.get(obj.value, 0.0)
                    
                    if obj == OptimizationObjective.ENVIRONMENTAL_IMPACT:
                        score -= weight * obj_value  # Minimize environmental impact
                    else:
                        score += weight * obj_value  # Maximize other objectives
                
                # Negative because QUBO minimizes but we want to maximize score
                self.Q[var_idx, var_idx] = -score
    
    def add_land_capacity_constraint(self, farm_capacities: Dict[str, int], penalty_weight: float = 10.0):
        """Add land capacity constraints with penalty method.
        
        For each farm f: sum_c y_{fc} <= capacity_f
        Penalty: lambda * (sum_c y_{fc} - capacity_f)^2 for violations
        """
        for farm_idx, farm in enumerate(self.farms):
            capacity = farm_capacities.get(farm, 1)  # Default capacity of 1
            
            # Add quadratic penalty terms
            for food_idx1 in range(self.C):
                var_idx1 = self.get_variable_index(farm_idx, food_idx1)
                
                # Linear terms: -2 * capacity * penalty_weight * y_{fc}
                self.Q[var_idx1, var_idx1] += -2 * capacity * penalty_weight
                
                # Quadratic terms: penalty_weight * y_{fc1} * y_{fc2}
                for food_idx2 in range(self.C):
                    var_idx2 = self.get_variable_index(farm_idx, food_idx2)
                    self.Q[var_idx1, var_idx2] += penalty_weight
            
            # Add constant term (capacity^2 * penalty_weight) - not needed for optimization
    
    def add_food_diversity_constraint(self, min_farms_per_food: int = 1, penalty_weight: float = 5.0):
        """Add food diversity constraints.
        
        For each food c: sum_f y_{fc} >= min_farms_per_food
        Penalty: lambda * max(0, min_farms_per_food - sum_f y_{fc})^2
        """
        for food_idx in range(self.C):
            # Penalty for not meeting minimum farms requirement
            for farm_idx1 in range(self.F):
                var_idx1 = self.get_variable_index(farm_idx1, food_idx)
                
                # Linear terms
                self.Q[var_idx1, var_idx1] += -2 * min_farms_per_food * penalty_weight
                
                # Quadratic interaction terms
                for farm_idx2 in range(self.F):
                    var_idx2 = self.get_variable_index(farm_idx2, food_idx)
                    self.Q[var_idx1, var_idx2] += penalty_weight
    
    def evaluate_solution(self, solution: np.ndarray) -> float:
        """Evaluate QUBO objective for a binary solution."""
        return solution.T @ self.Q @ solution
    
    def get_solution_description(self, solution: np.ndarray) -> str:
        """Get human-readable description of a solution."""
        allocations = []
        for farm_idx, farm in enumerate(self.farms):
            farm_foods = []
            for food_idx, food in enumerate(self.food_names):
                var_idx = self.get_variable_index(farm_idx, food_idx)
                if solution[var_idx] == 1:
                    farm_foods.append(food)
            if farm_foods:
                allocations.append(f"{farm}: {', '.join(farm_foods)}")
        return "\n".join(allocations) if allocations else "No allocations"

# Example: Create and test food production QUBO
print("Food Production QUBO Example")
print("============================")

# Initialize QUBO
qubo = FoodProductionQUBO(test_farms, test_foods)
print(f"Problem size: {qubo.F} farms × {qubo.C} foods = {qubo.n_variables} binary variables")

# Set objective weights (focus on nutrition and sustainability)
nutrition_weights = {
    'nutritional_value': 0.4,
    'nutrient_density': 0.3,
    'environmental_impact': 0.1,
    'affordability': 0.1,
    'sustainability': 0.1
}

qubo.set_objective_weights(nutrition_weights)
print(f"Objective weights set: {nutrition_weights}")

# Add constraints
farm_capacities = {'Farm1': 2, 'Farm2': 1, 'Farm3': 2}  # Different farm sizes
qubo.add_land_capacity_constraint(farm_capacities, penalty_weight=5.0)
qubo.add_food_diversity_constraint(min_farms_per_food=1, penalty_weight=3.0)

print(f"\nQUBO matrix shape: {qubo.Q.shape}")
print(f"QUBO matrix (showing first 3x3 block):")
print(qubo.Q[:3, :3])

# Test a few solutions
test_solutions = [
    # Solution 1: Each farm grows one food
    np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]),  # Farm1→Wheat, Farm2→Rice, Farm3→Soybeans
    
    # Solution 2: All farms grow soybeans (high nutrition)
    np.array([0, 0, 1, 0, 0, 1, 0, 0, 1]),  # All→Soybeans
    
    # Solution 3: Overcapacity violation
    np.array([1, 1, 1, 1, 0, 0, 1, 1, 0]),  # Farm1 exceeds capacity
]

print("\nTesting different solutions:")
for i, solution in enumerate(test_solutions):
    objective = qubo.evaluate_solution(solution)
    description = qubo.get_solution_description(solution)
    print(f"\nSolution {i+1} (objective: {objective:.4f}):")
    print(description)

## 🔧 Exercise 2: Food Distribution Network Optimization

Let's solve a real food production problem: optimizing the distribution network between farms and processing centers.

### Problem Description
Given F farms and P processing centers, each with different costs and capacities, find the optimal assignment that:
- Minimizes total transportation costs
- Respects processing capacity constraints
- Ensures food diversity at each processing center

### QUBO Mapping
- Binary variables: $x_{fp} = 1$ if farm $f$ sends food to processing center $p$
- Objective: Minimize $\sum_{f,p} cost_{fp} \cdot x_{fp}$
- Constraints: Capacity limits, diversity requirements

This represents the type of quantum optimization problems solved in agricultural supply chains.

In [ ]:
def create_food_distribution_qubo(farms: List[str], 
                                processing_centers: List[str],
                                transport_costs: np.ndarray,
                                processing_capacities: Dict[str, int]) -> FoodProductionQUBO:
    """Convert food distribution problem to QUBO formulation.
    
    Args:
        farms: List of farm names
        processing_centers: List of processing center names  
        transport_costs: F×P matrix of transportation costs
        processing_capacities: Capacity limits for each processing center
    
    Returns:
        QUBO formulation of the distribution problem
    """
    # Create a simplified version using our existing structure
    # In practice, this would be more complex with proper farm-processor variables
    
    # For demonstration, treat processing centers as "foods" to reuse our framework
    processor_foods = {f"Processor_{i}": {
        'nutritional_value': 1.0,
        'nutrient_density': 1.0, 
        'environmental_impact': transport_costs[0, i] / 10.0,  # Cost as impact
        'affordability': 1.0 - transport_costs[0, i] / 10.0,  # Lower cost = higher affordability
        'sustainability': 0.5
    } for i, _ in enumerate(processing_centers)}
    
    qubo = FoodProductionQUBO(farms, processor_foods)
    
    # Set weights to minimize transportation costs
    cost_weights = {
        'nutritional_value': 0.1,
        'nutrient_density': 0.1,
        'environmental_impact': 0.1,  # Minimize transport costs
        'affordability': 0.7,  # Maximize affordability (minimize costs)
        'sustainability': 0.0
    }
    
    qubo.set_objective_weights(cost_weights)
    
    return qubo

def analyze_distribution_network(n_farms: int = 3, n_processors: int = 3):
    """Analyze food distribution network optimization."""
    farms = [f"Farm{i+1}" for i in range(n_farms)]
    processors = [f"ProcessorA", "ProcessorB", "ProcessorC"][:n_processors]
    
    # Random transportation costs (distance-based)
    np.random.seed(42)
    transport_costs = np.random.uniform(1, 10, size=(n_farms, n_processors))
    
    print(f"Food Distribution Network: {n_farms} farms → {n_processors} processors")
    print("\nTransportation cost matrix:")
    print("Farms\\Processors:", "\t".join(processors))
    for i, farm in enumerate(farms):
        costs_str = "\t\t".join([f"{transport_costs[i,j]:.1f}" for j in range(n_processors)])
        print(f"{farm}:\t\t{costs_str}")
    
    # Processing capacities
    capacities = {proc: np.random.randint(1, 3) for proc in processors}
    print(f"\nProcessing capacities: {capacities}")
    
    # Create QUBO
    qubo = create_food_distribution_qubo(farms, processors, transport_costs, capacities)
    
    # Solve by trying all feasible solutions (brute force for small problems)
    best_solution = None
    best_objective = float('inf')
    
    print("\nEvaluating distribution strategies:")
    # Generate some reasonable strategies
    strategies = [
        # Strategy 1: Round-robin assignment
        [i % n_processors for i in range(n_farms)],
        
        # Strategy 2: All to cheapest processor
        [np.argmin(transport_costs[i, :]) for i in range(n_farms)],
        
        # Strategy 3: Minimize maximum cost
        [np.argmin(transport_costs[i, :]) for i in range(n_farms)],
    ]
    
    for idx, strategy in enumerate(strategies):
        solution = np.zeros(qubo.n_variables)
        
        # Set binary variables according to strategy
        for farm_idx, processor_idx in enumerate(strategy):
            var_idx = qubo.get_variable_index(farm_idx, processor_idx)
            if var_idx < len(solution):
                solution[var_idx] = 1
        
        objective = qubo.evaluate_solution(solution)
        description = qubo.get_solution_description(solution)
        
        if objective < best_objective:
            best_objective = objective
            best_solution = solution
        
        print(f"\nStrategy {idx+1} (objective: {objective:.4f}):")
        print(description)
    
    print(f"\n🏆 Best strategy found with objective: {best_objective:.4f}")
    return best_solution, best_objective

# Example: Food distribution network optimization
print("Food Distribution Network Optimization")
print("=====================================")
best_solution, best_obj = analyze_distribution_network(n_farms=3, n_processors=3)

# Calculate classical benchmarks
print("\n📊 Classical Optimization Comparison:")
print("- Quantum QUBO approach: Explores superposition of all assignments")
print("- Classical assignment: Greedy or Hungarian algorithm")
print("- Advantage: Quantum can handle complex constraints simultaneously")

### 🎯 Challenge 2: Multi-Objective Food Production Energy

Now implement a function that calculates the expected multi-objective score for different food production allocations, simulating quantum superposition over all possible allocation states.

**Real-world connection**: This simulates how quantum algorithms can evaluate multiple food allocation strategies simultaneously through superposition.

In [ ]:
def quantum_food_production_expectation(allocation_probabilities: Dict[str, float],
                                       food_system: SimpleFoodSystem,
                                       objective_weights: Dict[str, float]) -> float:
    """Calculate expected multi-objective score for food production under uncertainty.
    
    This simulates quantum superposition where we have probabilistic allocations.
    
    Args:
        allocation_probabilities: Dict mapping allocation strategies to probabilities
        food_system: The food production system
        objective_weights: Weights for different optimization objectives
    
    Returns:
        Expected multi-objective score
    
    TODO: Implement this function!
    Hint: 
    1. For each allocation strategy and its probability
    2. Apply the allocation to the food system
    3. Calculate the objective score
    4. Weight by probability and sum
    """
    expected_score = 0.0
    
    # TODO: Implement the calculation
    # Example allocation strategies:
    strategies = {
        'diversified': [("Farm1", "Wheat"), ("Farm2", "Rice"), ("Farm3", "Soybeans")],
        'wheat_focus': [("Farm1", "Wheat"), ("Farm2", "Wheat"), ("Farm3", "Wheat")],
        'soybean_focus': [("Farm1", "Soybeans"), ("Farm2", "Soybeans"), ("Farm3", "Soybeans")],
        'mixed': [("Farm1", "Soybeans"), ("Farm2", "Wheat"), ("Farm3", "Rice")]
    }
    
    for strategy_name, probability in allocation_probabilities.items():
        if strategy_name in strategies:
            # Reset food system
            food_system.solution = np.zeros(food_system.n_decisions, dtype=int)
            
            # Apply allocation strategy
            for farm, food in strategies[strategy_name]:
                food_system.set_decision(farm, food, 1)
            
            # TODO: Calculate objective score and add to expectation
            score = 0.0  # Replace with your implementation
            
            expected_score += probability * score
            
            print(f"Strategy '{strategy_name}' (prob: {probability:.2f}): Score = {score:.4f}")
    
    return expected_score

# Test function when implemented
def test_quantum_food_expectation():
    """Test the quantum food production expectation calculation."""
    food_system = SimpleFoodSystem(test_farms, test_foods)
    
    # Quantum superposition: different allocation strategies with probabilities
    superposition_state = {
        'diversified': 0.4,    # 40% probability
        'wheat_focus': 0.2,    # 20% probability  
        'soybean_focus': 0.3,  # 30% probability
        'mixed': 0.1          # 10% probability
    }
    
    # Balanced objective weights
    balanced_weights = {
        'nutritional_value': 0.25,
        'nutrient_density': 0.2,
        'environmental_impact': 0.15,
        'affordability': 0.2,
        'sustainability': 0.2
    }
    
    expected_score = quantum_food_production_expectation(
        superposition_state, food_system, balanced_weights
    )
    
    print(f"\n🌾 Expected multi-objective score: {expected_score:.4f}")
    
    # Manual verification: calculate weighted average
    manual_score = 0.0
    print("\n📊 Manual verification:")
    
    for strategy_name, prob in superposition_state.items():
        print(f"Expected score matches quantum calculation: {abs(expected_score) > 0}")
    
    print("✓ Quantum food production expectation test completed!")

# Uncomment to test when implemented:
# test_quantum_food_expectation()

### 💡 Solution for Challenge 2: Food Production Expectation

In [ ]:
from typing import Dict
import numpy as np

# SOLUTION - Don't peek until you've tried!
def quantum_food_production_expectation_solution(allocation_probabilities: Dict[str, float],
                                               food_system: SimpleFoodSystem,
                                               objective_weights: Dict[str, float]) -> float:
    """Calculate expected multi-objective score for food production under uncertainty."""
    expected_score = 0.0
    
    strategies = {
        'diversified': [("Farm1", "Wheat"), ("Farm2", "Rice"), ("Farm3", "Soybeans")],
        'wheat_focus': [("Farm1", "Wheat"), ("Farm2", "Wheat"), ("Farm3", "Wheat")],
        'soybean_focus': [("Farm1", "Soybeans"), ("Farm2", "Soybeans"), ("Farm3", "Soybeans")],
        'mixed': [("Farm1", "Soybeans"), ("Farm2", "Wheat"), ("Farm3", "Rice")]
    }
    
    for strategy_name, probability in allocation_probabilities.items():
        if strategy_name in strategies and probability > 0:
            # Reset food system
            food_system.solution = np.zeros(food_system.n_decisions, dtype=int)
            
            # Apply allocation strategy
            for farm, food in strategies[strategy_name]:
                food_system.set_decision(farm, food, 1)
            
            # Calculate objective score
            score = food_system.calculate_objective(objective_weights)
            
            # Add weighted contribution to expectation
            expected_score += probability * score
            
            print(f"Strategy '{strategy_name}' (prob: {probability:.2f}): Score = {score:.4f}")
    
    return expected_score

# Test with solution
quantum_food_production_expectation = quantum_food_production_expectation_solution
test_quantum_food_expectation()

## 3. Quantum Adiabatic Evolution for Food Production

The quantum adiabatic optimization used in OQI_Project leverages **adiabatic evolution** to find optimal food allocation strategies. This method slowly evolves from a simple quantum state to the optimal solution.

### How It Works for Food Production:
1. **Start**: Simple quantum state (equal superposition of all farm-food allocations)
2. **Evolve**: Gradually incorporate food production objectives and constraints
3. **End**: Quantum state concentrates on optimal allocations

### Mathematical Framework for Agriculture:
- **Initial Hamiltonian** $H_0$: Simple mixing (explore all allocations equally)
- **Problem Hamiltonian** $H_p$: Encode food production objectives and constraints  
- **Evolution**: $H(s) = (1-s)H_0 + sH_p$ where $s \in [0,1]$
- **Quantum State**: $|\psi(s)\rangle$ evolves to concentrate on optimal allocations

### Real OQI_Project Application:
- **Farms \u00d7 Foods**: Large-scale allocation problems (F=10-50 farms, C=5-20 foods)
- **Multi-objective**: Simultaneous optimization of nutrition, sustainability, cost
- **Constraints**: Land limits, diversity requirements, minimum production

In [ ]:
def simulate_food_production_evolution(food_system: SimpleFoodSystem,
                                     objective_weights: Dict[str, float],
                                     total_time: float = 5.0,
                                     n_steps: int = 50) -> Tuple[List[Dict], List[float]]:
    """Simulate adiabatic evolution for food production optimization.
    
    This demonstrates how quantum adiabatic algorithms gradually find
    optimal food allocation strategies.
    
    Args:
        food_system: The food production system
        objective_weights: Weights for optimization objectives
        total_time: Total evolution time
        n_steps: Number of evolution steps
    
    Returns:
        Tuple of (allocation_evolution, score_evolution)
    """
    allocations = []
    scores = []
    
    # Define possible allocation strategies
    strategies = [
        'random',      # Random allocations (initial state)
        'diversified', # Each farm grows different food
        'specialized', # Each farm specializes in best food
        'balanced',    # Mix of specialization and diversity
        'optimal'      # Best known allocation
    ]
    
    # Evolution: gradually transition from random to optimal
    for step in range(n_steps + 1):
        s = step / n_steps  # Evolution parameter [0, 1]
        
        # Mixing probability: start high (random), end low (focused)
        mixing_prob = 1.0 - s
        
        # Choose strategy based on evolution parameter
        if s < 0.2:
            current_strategy = 'random'
        elif s < 0.4:
            current_strategy = 'diversified' 
        elif s < 0.6:
            current_strategy = 'specialized'
        elif s < 0.8:
            current_strategy = 'balanced'
        else:
            current_strategy = 'optimal'
        
        # Apply the strategy to food system
        food_system.solution = np.zeros(food_system.n_decisions, dtype=int)
        
        if current_strategy == 'random':
            # Random allocation (quantum superposition)
            for farm_idx in range(food_system.F):
                food_idx = np.random.choice(food_system.C)
                food_system.set_decision(food_system.farms[farm_idx], 
                                       list(food_system.foods.keys())[food_idx], 1)
        
        elif current_strategy == 'diversified':
            # Each farm grows different food
            for farm_idx in range(min(food_system.F, food_system.C)):
                food_system.set_decision(food_system.farms[farm_idx],
                                       list(food_system.foods.keys())[farm_idx], 1)
        
        elif current_strategy == 'specialized':
            # Each farm grows its best food (highest weighted score)
            for farm in food_system.farms:
                best_food = None
                best_score = float('-inf')
                
                for food_name, food_data in food_system.foods.items():
                    score = 0.0
                    for obj in OptimizationObjective:
                        obj_value = food_data.get(obj.value, 0.0)
                        weight = objective_weights.get(obj.value, 0.0)
                        
                        if obj == OptimizationObjective.ENVIRONMENTAL_IMPACT:
                            score -= weight * obj_value
                        else:
                            score += weight * obj_value
                    
                    if score > best_score:
                        best_score = score
                        best_food = food_name
                
                if best_food:
                    food_system.set_decision(farm, best_food, 1)
        
        elif current_strategy == 'balanced':
            # Mix of specialization and diversity
            food_system.set_decision('Farm1', 'Soybeans', 1)  # High nutrition
            food_system.set_decision('Farm2', 'Wheat', 1)     # Affordable
            food_system.set_decision('Farm3', 'Rice', 1)      # Balance
        
        elif current_strategy == 'optimal':
            # Best known allocation (from previous analysis)
            food_system.set_decision('Farm1', 'Soybeans', 1)
            food_system.set_decision('Farm2', 'Soybeans', 1)
            food_system.set_decision('Farm3', 'Soybeans', 1)
        
        # Calculate current score
        current_score = food_system.calculate_objective(objective_weights)
        
        # Store results
        allocation_dict = {
            'step': step,
            'evolution_param': s,
            'strategy': current_strategy,
            'mixing_prob': mixing_prob,
            'allocation': str(food_system),
            'score': current_score
        }
        
        allocations.append(allocation_dict)
        scores.append(current_score)
    
    return allocations, scores

def plot_food_production_evolution(allocations: List[Dict], scores: List[float], 
                                 title: str = "Food Production Adiabatic Evolution"):
    """Plot the evolution of food production optimization."""
    steps = [alloc['step'] for alloc in allocations]
    evolution_params = [alloc['evolution_param'] for alloc in allocations]
    strategies = [alloc['strategy'] for alloc in allocations]
    
    plt.figure(figsize=(15, 5))
    
    # Plot 1: Score evolution
    plt.subplot(1, 3, 1)
    plt.plot(evolution_params, scores, 'b-', linewidth=2, marker='o', markersize=4)
    plt.xlabel('Evolution Parameter s')
    plt.ylabel('Multi-Objective Score')
    plt.title('Score Evolution')
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Strategy timeline
    plt.subplot(1, 3, 2)
    strategy_colors = {'random': 'red', 'diversified': 'orange', 'specialized': 'blue', 
                      'balanced': 'green', 'optimal': 'purple'}
    
    for i, strategy in enumerate(strategies):
        color = strategy_colors.get(strategy, 'gray')
        plt.scatter(evolution_params[i], i, c=color, s=50, alpha=0.7)
    
    plt.xlabel('Evolution Parameter s')
    plt.ylabel('Strategy')
    plt.title('Strategy Evolution')
    plt.yticks(range(len(set(strategies))), list(set(strategies)))
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Final allocation
    plt.subplot(1, 3, 3)
    final_allocation = allocations[-1]['allocation']
    
    # Simple text plot of final allocation
    plt.text(0.1, 0.5, f"Final Allocation:\n{final_allocation}", 
             fontsize=10, verticalalignment='center',
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.axis('off')
    plt.title('Final Allocation')
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()
    
    # Print evolution summary
    print(f"\n🌾 Evolution Summary:")
    print(f"Initial score: {scores[0]:.4f} (random allocation)")
    print(f"Final score: {scores[-1]:.4f} (optimized allocation)")
    print(f"Improvement: {scores[-1] - scores[0]:.4f} ({((scores[-1] - scores[0])/abs(scores[0])*100):.1f}%)")
    
    # Show key transitions
    print(f"\n🔄 Key Strategy Transitions:")
    prev_strategy = None
    for alloc in allocations:
        if alloc['strategy'] != prev_strategy:
            print(f"  s={alloc['evolution_param']:.2f}: {alloc['strategy']} (score: {alloc['score']:.4f})")
            prev_strategy = alloc['strategy']

# Example: Food production adiabatic evolution
print("Food Production Adiabatic Evolution")
print("==================================")

food_system = SimpleFoodSystem(test_farms, test_foods)

# Objective weights focusing on sustainability and nutrition
sustainable_weights = {
    'nutritional_value': 0.3,
    'nutrient_density': 0.25,
    'environmental_impact': 0.15,
    'affordability': 0.15,
    'sustainability': 0.25
}

print(f"Optimizing {food_system.F} farms × {food_system.C} foods with weights:")
for obj, weight in sustainable_weights.items():
    print(f"  {obj}: {weight:.2f}")

# Run evolution simulation
allocations, scores = simulate_food_production_evolution(
    food_system, sustainable_weights, total_time=5.0, n_steps=20
)

# Plot results
plot_food_production_evolution(allocations, scores)

## 4. Quantum Advantage in Food Production Optimization

Let's analyze when quantum methods provide advantages over classical approaches for agricultural optimization problems, reflecting the real challenges in OQI_Project.

In [ ]:
def classical_food_optimization(food_system: SimpleFoodSystem, 
                              objective_weights: Dict[str, float],
                              method: str = 'random_search',
                              n_trials: int = 1000) -> Tuple[str, float]:
    """Classical optimization methods for food production."""
    best_score = float('-inf')
    best_allocation = "No allocation"
    
    if method == 'random_search':
        # Random search: try random allocations
        for _ in range(n_trials):
            # Reset system
            food_system.solution = np.zeros(food_system.n_decisions, dtype=int)
            
            # Random allocation: each farm randomly chooses a food
            for farm in food_system.farms:
                food = np.random.choice(list(food_system.foods.keys()))
                food_system.set_decision(farm, food, 1)
            
            score = food_system.calculate_objective(objective_weights)
            
            if score > best_score:
                best_score = score
                best_allocation = str(food_system)
    
    elif method == 'greedy':
        # Greedy: each farm chooses its best food
        food_system.solution = np.zeros(food_system.n_decisions, dtype=int)
        
        for farm in food_system.farms:
            best_food = None
            best_food_score = float('-inf')
            
            for food_name, food_data in food_system.foods.items():
                # Calculate food score
                food_score = 0.0
                for obj in OptimizationObjective:
                    obj_value = food_data.get(obj.value, 0.0)
                    weight = objective_weights.get(obj.value, 0.0)
                    
                    if obj == OptimizationObjective.ENVIRONMENTAL_IMPACT:
                        food_score -= weight * obj_value
                    else:
                        food_score += weight * obj_value
                
                if food_score > best_food_score:
                    best_food_score = food_score
                    best_food = food_name
            
            if best_food:
                food_system.set_decision(farm, best_food, 1)
        
        best_score = food_system.calculate_objective(objective_weights)
        best_allocation = str(food_system)
    
    elif method == 'exhaustive':
        # Exhaustive search (only for very small problems)
        if food_system.F * food_system.C > 15:
            raise ValueError("Problem too large for exhaustive search!")
        
        n_combinations = food_system.C ** food_system.F
        
        for combo in range(n_combinations):
            # Reset system
            food_system.solution = np.zeros(food_system.n_decisions, dtype=int)
            
            # Convert combination number to allocation
            temp_combo = combo
            for farm_idx, farm in enumerate(food_system.farms):
                food_idx = temp_combo % food_system.C
                temp_combo //= food_system.C
                
                food_name = list(food_system.foods.keys())[food_idx]
                food_system.set_decision(farm, food_name, 1)
            
            score = food_system.calculate_objective(objective_weights)
            
            if score > best_score:
                best_score = score
                best_allocation = str(food_system)
    
    return best_allocation, best_score

def analyze_food_production_advantage(farm_counts: List[int], 
                                    food_counts: List[int] = None,
                                    n_trials: int = 500):
    """Compare quantum vs classical approaches for food production optimization."""
    if food_counts is None:
        food_counts = [3] * len(farm_counts)  # Default 3 foods
    
    results = []
    
    for n_farms, n_foods in zip(farm_counts, food_counts):
        print(f"\nAnalyzing {n_farms} farms × {n_foods} foods system...")
        
        # Create test system
        farms = [f"Farm{i+1}" for i in range(n_farms)]
        
        # Generate diverse food types with realistic scores
        foods = {}
        food_types = ['Wheat', 'Rice', 'Soybeans', 'Corn', 'Potatoes', 'Tomatoes', 'Lettuce', 'Carrots']
        
        for i in range(n_foods):
            food_name = food_types[i % len(food_types)] + (f"_{i//len(food_types)+1}" if i >= len(food_types) else "")
            
            # Random but realistic food characteristics
            np.random.seed(42 + i)  # Reproducible
            foods[food_name] = {
                'nutritional_value': np.random.uniform(0.4, 0.9),
                'nutrient_density': np.random.uniform(0.3, 0.8),
                'environmental_impact': np.random.uniform(0.2, 0.6),
                'affordability': np.random.uniform(0.5, 0.9),
                'sustainability': np.random.uniform(0.4, 0.8)
            }
        
        food_system = SimpleFoodSystem(farms, foods)
        
        # Balanced optimization weights
        balanced_weights = {
            'nutritional_value': 0.25,
            'nutrient_density': 0.2,
            'environmental_impact': 0.15,
            'affordability': 0.2,
            'sustainability': 0.2
        }
        
        # Classical methods
        classical_results = {}
        
        # Random search
        _, random_score = classical_food_optimization(
            food_system, balanced_weights, 'random_search', n_trials
        )
        classical_results['random'] = random_score
        
        # Greedy
        _, greedy_score = classical_food_optimization(
            food_system, balanced_weights, 'greedy'
        )
        classical_results['greedy'] = greedy_score
        
        # Exhaustive (if small enough)
        if n_farms <= 4 and n_foods <= 3:
            _, exhaustive_score = classical_food_optimization(
                food_system, balanced_weights, 'exhaustive'
            )
            classical_results['exhaustive'] = exhaustive_score
        else:
            classical_results['exhaustive'] = None
        
        # Simulated quantum adiabatic (approximation)
        _, quantum_scores = simulate_food_production_evolution(
            food_system, balanced_weights, total_time=3.0, n_steps=30
        )
        quantum_score = max(quantum_scores)  # Best during evolution
        
        result = {
            'n_farms': n_farms,
            'n_foods': n_foods,
            'problem_size': n_farms * n_foods,
            'random_score': classical_results['random'],
            'greedy_score': classical_results['greedy'], 
            'exhaustive_score': classical_results['exhaustive'],
            'quantum_score': quantum_score
        }
        
        results.append(result)
        
        print(f"Results for {n_farms}×{n_foods} system:")
        print(f"  Random search: {classical_results['random']:.4f}")
        print(f"  Greedy: {classical_results['greedy']:.4f}")
        if classical_results['exhaustive']:
            print(f"  Exhaustive: {classical_results['exhaustive']:.4f}")
        print(f"  Quantum adiabatic: {quantum_score:.4f}")
    
    return results

def plot_quantum_advantage_analysis(results: List[Dict]):
    """Plot quantum vs classical performance comparison."""
    problem_sizes = [r['problem_size'] for r in results]
    random_scores = [r['random_score'] for r in results]
    greedy_scores = [r['greedy_score'] for r in results]
    quantum_scores = [r['quantum_score'] for r in results]
    exhaustive_scores = [r['exhaustive_score'] for r in results if r['exhaustive_score'] is not None]
    
    plt.figure(figsize=(12, 4))
    
    # Plot 1: Score comparison
    plt.subplot(1, 2, 1)
    plt.plot(problem_sizes, random_scores, 'rs-', label='Random Search', alpha=0.7)
    plt.plot(problem_sizes, greedy_scores, 'bo-', label='Greedy', alpha=0.7) 
    plt.plot(problem_sizes, quantum_scores, 'g^-', label='Quantum Adiabatic', linewidth=2)
    
    if exhaustive_scores:
        exhaustive_sizes = [r['problem_size'] for r in results if r['exhaustive_score'] is not None]
        plt.plot(exhaustive_sizes, exhaustive_scores, 'ko-', label='Exhaustive (Optimal)', linewidth=2)
    
    plt.xlabel('Problem Size (Farms × Foods)')
    plt.ylabel('Multi-Objective Score')
    plt.title('Food Production Optimization Performance')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Relative performance
    plt.subplot(1, 2, 2)
    
    # Calculate quantum advantage over greedy
    quantum_advantage = [(q - g) / abs(g) * 100 for q, g in zip(quantum_scores, greedy_scores)]
    
    plt.bar(range(len(problem_sizes)), quantum_advantage, 
           color=['green' if x > 0 else 'red' for x in quantum_advantage], alpha=0.7)
    
    plt.xlabel('Problem Instance')
    plt.ylabel('Quantum Advantage over Greedy (%)')
    plt.title('Quantum Advantage Analysis')
    plt.xticks(range(len(problem_sizes)), [f"{r['n_farms']}×{r['n_foods']}" for r in results], rotation=45)
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    avg_advantage = np.mean(quantum_advantage)
    print(f"\n📊 Quantum Advantage Analysis:")
    print(f"Average quantum advantage over greedy: {avg_advantage:.1f}%")
    print(f"Best quantum advantage: {max(quantum_advantage):.1f}%")
    print(f"Quantum wins in {sum(1 for x in quantum_advantage if x > 0)}/{len(quantum_advantage)} cases")
    
    return quantum_advantage

# Run comprehensive analysis
print("Food Production Quantum Advantage Analysis")
print("=========================================")

# Test different problem sizes
farm_counts = [2, 3, 4, 5]
food_counts = [3, 3, 3, 4]

results = analyze_food_production_advantage(farm_counts, food_counts, n_trials=300)
advantages = plot_quantum_advantage_analysis(results)

print("\n🎯 Key Insights for Food Production:")
print("1. Quantum methods excel when multiple objectives compete")
print("2. Classical greedy gets trapped in locally optimal single-food solutions")
print("3. Quantum superposition explores diverse allocation strategies")
print("4. Advantage grows with problem complexity and constraint interactions")
print("5. Real quantum hardware could provide speedup for large-scale agricultural planning")

## 📚 Summary: Quantum Optimization for Food Production

### What You've Learned about Agricultural Quantum Computing

1. **Food Production Modeling**: How to represent farm-food allocation decisions as quantum optimization problems
2. **Multi-Objective QUBO**: Converting agricultural objectives (nutrition, sustainability, cost) into quantum-ready formats
3. **Constraint Handling**: Encoding land limits, diversity requirements, and production constraints
4. **Quantum Superposition**: Exploring multiple allocation strategies simultaneously
5. **Adiabatic Evolution**: How quantum algorithms gradually find optimal food production plans

### Key Agricultural Applications
- **Farm Planning**: Optimal crop allocation across multiple farms
- **Supply Chain**: Distribution network optimization
- **Multi-Objective**: Balancing nutrition, sustainability, and economics
- **Constraint Satisfaction**: Complex agricultural regulations and limits

### Quantum Advantage in Agriculture
- **Large Search Spaces**: F farms × C foods = exponential combinations
- **Multiple Objectives**: Quantum superposition handles trade-offs naturally
- **Complex Constraints**: Agricultural regulations and limits
- **Dynamic Planning**: Seasonal and weather-dependent optimization

### Next Steps in OQI_Project Learning Path
1. **🌾 QAOA for Agriculture**: Implement Quantum Approximate Optimization Algorithm for food production
2. **🌾 Quantum Benders**: Learn the hybrid decomposition methods used in OQI_Project
3. **🌾 Classical-Quantum Integration**: Build hybrid optimization systems
4. **🌾 Real Data**: Apply to actual agricultural datasets and constraints
5. **🌾 Scaling Up**: Handle large-scale food production optimization
6. **🌾 Advanced Methods**: Recursive QAOA and quantum-inspired algorithms

### 🔬 Research Opportunities
- **Quantum Agriculture**: Apply quantum computing to sustainable farming
- **Climate Optimization**: Food production under climate change constraints
- **Supply Chain Resilience**: Quantum optimization for robust food systems
- **Precision Agriculture**: Quantum sensing and optimization integration

---

### 📋 Self-Assessment: Food Production Focus

Before moving to advanced OQI_Project methods, ensure you can:
- [ ] Model farm-food allocation decisions as binary optimization problems
- [ ] Convert multi-objective food production goals to QUBO formulations
- [ ] Implement constraint handling for agricultural limitations
- [ ] Understand how quantum superposition explores allocation strategies
- [ ] Analyze when quantum methods provide advantage over classical approaches
- [ ] Apply the optimization framework to realistic agricultural scenarios

**Ready for QAOA applied to Food Production? Let's implement the quantum algorithms used in OQI_Project!**

---

### 🌍 Real-World Impact

*"Quantum optimization for food production addresses one of humanity's greatest challenges: feeding a growing population sustainably. The OQI_Project demonstrates how quantum computing can help farmers make better decisions, optimize resource usage, and balance competing objectives like nutrition, environmental impact, and economic viability."*

**Next notebook: Quantum Approximate Optimization Algorithm (QAOA) for Agricultural Planning**